# Install

In [ ]:
!pip3 install hydra-core

In [ ]:
!pip3 install transformers

In [ ]:
!pip3 install datasets

In [ ]:
!pip3 install pshmodule

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Run

In [1]:
import os
os.chdir('drive/MyDrive/MemeProject/src/sbert')

In [6]:
!python3 ref_emb.py

/content/drive/MyDrive/MemeProject/src/sbert/ref_emb.py:9: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="./", config_name="config")
/usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
data shape : (181672, 3)
embeddings : tensor([[-0.8169,  0.3020, -1.0084,  ...,  0.0834, -0.6546, -1.4096],
        [-0.8169,  0.3020, -1.0084,  ...,  0.0834, -0.6546, -1.4096],
        [-0.8169,  0.3020, -1.0084,  ...,  0.0834, -0.6546, -1.4096]])


# Predict

In [ ]:
import os
os.chdir('drive/MyDrive/MemeProject/src/sbert')

In [20]:
!python3 predict.py

/content/drive/MyDrive/MemeProject/src/sbert/predict.py:10: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="./", config_name="config")
/usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
코사인 유사도 연산 결과 : [90391 90389 90388 90392]
입력 문장 : 너... 진짜 짜증나게 하지마라
유사 문장 : 아씨 짜증나게 하네
밈 답변 : 후덜덜...! 제발 그만해~


In [ ]:
|